In [1]:
from llama_index.core import download_loader
from pathlib import Path
from llama_index.core.node_parser import SimpleFileNodeParser, SimpleNodeParser
import os
import logging
import sys
from llama_index.core.callbacks import (
    CallbackManager
)
from llama_index.core import (SimpleDirectoryReader, 
                              ServiceContext,
                              VectorStoreIndex, 
                              )
from llama_index.core.retrievers import (VectorIndexRetriever, 
                                         BaseRetriever)


from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.schema import (QueryBundle, 
                                     QueryType,
                                     NodeWithScore)
from typing import List
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.callbacks.wandb import WandbCallbackHandler
from llama_index.core import load_index_from_storage
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core import (
  VectorStoreIndex, 
  SimpleKeywordTableIndex, 
  SimpleDirectoryReader, 
  StorageContext,
  ServiceContext
 )
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1"

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/langchain/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
print("=" * 10 + "添加嵌入与LLM模型"  + "=" * 10)
def completion_to_prompt(completion):
   return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt

llm = HuggingFaceLLM(
    model_name="../model_local/qwen/Qwen1.5-7B-Chat",
    tokenizer_name="../model_local/qwen/Qwen1.5-7B-Chat",
    context_window=30000, 
    max_new_tokens=2000,
    generate_kwargs={"temperature": 0.7, 
                     "top_k": 50, 
                     "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto"
    )

embedding_model = "../embedding_model"
embed_model = HuggingFaceEmbedding(
    model_name = embedding_model
)


print("=" * 10 + "创建索引"  + "=" * 10)
"""
创建一个索引，使用向量存储来存储嵌入， 对该文档执行查询以获得相关或类似的文档
"""
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, 
    llm=llm
)

==========添加嵌入与LLM模型==========


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.62s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


==========创建索引==========


/tmp/ipykernel_192192/4168266345.py:45: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [4]:
# check if data indexes already exists
if not os.path.exists("./data_base/llama_rag_9"):
    documents = SimpleDirectoryReader(
        input_dir="../ops_txt_temp").load_data(show_progress=True)
    
    # create nodes parser
    node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

    # split_into node
    base_nodes = node_parser.get_nodes_from_documents(documents=documents)

    # create index
    index = VectorStoreIndex(nodes=base_nodes, 
                             service_context=service_context)
    
    index.storage_context.persist(persist_dir="./data_base/llama_rag_9")
else:
    # load existing index
    storage_context = StorageContext.from_defaults(
        persist_dir="./data_base/llama_rag_9")
    index = load_index_from_storage(storage_context=storage_context)

Loading files: 100%|██████████| 40344/40344 [00:04<00:00, 8499.00file/s]


In [2]:
# Import groundedness feedback function
from trulens_eval.feedback import GroundTruthAgreement, Groundedness
from trulens_eval import TruBasicApp, Feedback, Tru, Select

ImportError: cannot import name 'Groundedness' from 'trulens_eval.feedback' (/root/miniconda3/envs/langchain/lib/python3.10/site-packages/trulens_eval/feedback/__init__.py)